### Aggregation

##### 집계함수

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = {
    'name': ['김혜수', '박보검', '송혜교', '아이유', '공유', '전지현', '수지', '유재석', '강동원', '윤아'],
    'department': ['인사', '재무', '재무', 'IT', 'IT', '인사', '재무', '인사', 'IT', '재무'],
    'job': ['사원', '과장', '대리', '사원', '팀장', '대리', '대리', '과장', '사원', '팀장'],
    'branch': ['서울', '부산', '서울', '부산', '서울', '서울', '서울', '부산', '서울', '부산'],
    'salary': [3500, 5000, 4200, 3800, 6000, 4500, 4600, 5200, 3900, 6100],
    'hire_date': ['2020-03-15', '2018-07-10', '2019-05-20', '2021-01-10', '2017-09-25', '2019-03-05', '2020-06-12', '2016-11-30', '2022-04-01', '2017-12-15']
}

In [4]:
df = pd.DataFrame(data)
df['hire_date'] = pd.to_datetime(df['hire_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   name        10 non-null     object        
 1   department  10 non-null     object        
 2   job         10 non-null     object        
 3   branch      10 non-null     object        
 4   salary      10 non-null     int64         
 5   hire_date   10 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 612.0+ bytes


In [6]:
print(df.count())   # dataframe 레벨 | 갯수 집계

# Series 레벨 집계 함수
# 숫자형이여야 집계 가능한 함수들 -> 숫자형이 있는 행으로 설정해야 함
print(df['salary'].sum())
print(df['salary'].mean())
print(df['salary'].max(), df['salary'].min())
print(df['salary'].median())
print(df['salary'].var())
print(df['salary'].std())

name          10
department    10
job           10
branch        10
salary        10
hire_date     10
dtype: int64
46800
4680.0
6100 3500
4550.0
797333.3333333334
892.9352346801718


---
## groupby

- 특정 조건에 맞게 행을 그룹핑하는 메서드
- DataFrameGroupBy 객체 반환

In [8]:
groupby_dept = df.groupby('department')
print(groupby_dept)
print(type(groupby_dept))
print(groupby_dept.groups)

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
{'IT': [3, 4, 8], '인사': [0, 5, 7], '재무': [1, 2, 6, 9]}


In [10]:
# 그룹별 집계함수 조회
print(groupby_dept['salary'].mean())
print(groupby_dept['salary'].max())
print(groupby_dept['salary'].min())

department
IT    4566.666667
인사    4400.000000
재무    4975.000000
Name: salary, dtype: float64
department
IT    6000
인사    5200
재무    6100
Name: salary, dtype: int64
department
IT    3800
인사    3500
재무    4200
Name: salary, dtype: int64


In [11]:
# 숫자형 데이터가 아닌 경우 count
groupby_dept['job'].value_counts()

department  job
IT          사원     2
            팀장     1
인사          과장     1
            대리     1
            사원     1
재무          대리     2
            과장     1
            팀장     1
Name: count, dtype: int64

In [12]:
# 날짜형 데이터
print(groupby_dept['hire_date'].min())  # 가장 과거인 날짜 -> 가장 빠른 입사일
print(groupby_dept['hire_date'].max())  # 가장 최근인 날짜 -> 가장 늦은 입사일

department
IT   2017-09-25
인사   2016-11-30
재무   2017-12-15
Name: hire_date, dtype: datetime64[ns]
department
IT   2022-04-01
인사   2020-03-15
재무   2020-06-12
Name: hire_date, dtype: datetime64[ns]


In [13]:
# named aggregation : .agg(컬럼명=(어떤것으로 어떤것을 집계할 건지 순서대로 입력(튜플 형식)))
groupby_dept.agg(
    salary_mean = ('salary', 'mean'),
    salary_max = ('salary', 'max'),
    hire_date_min = ('hire_date', 'min'),
    hire_date_max = ('hire_date', 'max')
)

,salary_mean,salary_max,hire_date_min,hire_date_max
department,,,,
IT,4566.666667,6000,2017-09-25,2022-04-01
인사,4400.000000,5200,2016-11-30,2020-03-15
재무,4975.000000,6100,2017-12-15,2020-06-12


In [15]:
# 두 개 이상의 컬럼으로 그룹핑
groupby_dept_job = df.groupby(['department', 'job'])
print(groupby_dept_job.groups)

{('IT', '사원'): [3, 8], ('IT', '팀장'): [4], ('인사', '과장'): [7], ('인사', '대리'): [5], ('인사', '사원'): [0], ('재무', '과장'): [1], ('재무', '대리'): [2, 6], ('재무', '팀장'): [9]}


In [16]:
print(groupby_dept_job.count())

                name  branch  salary  hire_date
department job                                 
IT         사원      2       2       2          2
           팀장      1       1       1          1
인사         과장      1       1       1          1
           대리      1       1       1          1
           사원      1       1       1          1
재무         과장      1       1       1          1
           대리      2       2       2          2
           팀장      1       1       1          1


In [17]:
print(groupby_dept_job.value_counts())

department  job  name  branch  salary  hire_date 
IT          사원   강동원   서울      3900    2022-04-01    1
                 아이유   부산      3800    2021-01-10    1
            팀장   공유    서울      6000    2017-09-25    1
인사          과장   유재석   부산      5200    2016-11-30    1
            대리   전지현   서울      4500    2019-03-05    1
            사원   김혜수   서울      3500    2020-03-15    1
재무          과장   박보검   부산      5000    2018-07-10    1
            대리   송혜교   서울      4200    2019-05-20    1
                 수지    서울      4600    2020-06-12    1
            팀장   윤아    부산      6100    2017-12-15    1
Name: count, dtype: int64


In [18]:
for (name, group) in groupby_dept_job:
    print(name)
    display(group)      # jupyter 전용 함수

('IT', '사원')


,name,department,job,branch,salary,hire_date
3,아이유,IT,사원,부산,3800,2021-01-10
8,강동원,IT,사원,서울,3900,2022-04-01


('IT', '팀장')


,name,department,job,branch,salary,hire_date
4,공유,IT,팀장,서울,6000,2017-09-25


('인사', '과장')


...

,name,department,job,branch,salary,hire_date
1,박보검,재무,과장,부산,5000,2018-07-10


('재무', '대리')


,name,department,job,branch,salary,hire_date
2,송혜교,재무,대리,서울,4200,2019-05-20
6,수지,재무,대리,서울,4600,2020-06-12


('재무', '팀장')


,name,department,job,branch,salary,hire_date
9,윤아,재무,팀장,부산,6100,2017-12-15


---
### pivot_table

- 특정 컬럼(인덱스, 컬럼)을 기준으로 값 컬럼을 열람하는 기능
- 연관 있는 데이터를 가독성 좋게 출력 가능

In [19]:
data = {
    'Date': ['2024-10-11', '2024-10-12', '2024-10-13', '2024-10-14', '2024-10-14'],
    'City': ['서울', '인천', '평택', '화성', '서울'],
    'Temperature': [17, 18, 19, 18, 19]
}

df= pd.DataFrame(data)

In [21]:
df.pivot_table(index='Date', columns='City', values='Temperature')

City,서울,인천,평택,화성
Date,,,,
2024-10-11,17.0,NaN,NaN,NaN
2024-10-12,NaN,18.0,NaN,NaN
2024-10-13,NaN,NaN,19.0,NaN
2024-10-14,19.0,NaN,NaN,18.0


In [22]:
# 제품별 판매데이터 
data = {
    '날짜': [
        '2024-01-01', '2024-01-01', '2024-01-01', '2024-01-01',
        '2024-01-02', '2024-01-02', '2024-01-02', '2024-01-02',
        '2024-01-03', '2024-01-03'
    ],
    '상품': [
        '아이폰 14', '갤럭시 S23', '아이패드 프로', '아이폰 14',
        '아이폰 14', '갤럭시 S23', '아이패드 프로', '갤럭시 S23',
        '맥북 프로', '아이폰 14'
    ],
    '판매량': [15, 30, 5, 10, 20, 25, 10, 5, 8, 12],
    '가격': [1000, 900, 800, 1000, 1000, 900, 800, 900, 2000, 1000]
}

df = pd.DataFrame(data)
df

,날짜,상품,판매량,가격
0,2024-01-01,아이폰 14,15,1000
1,2024-01-01,갤럭시 S23,30,900
2,2024-01-01,아이패드 프로,5,800
3,2024-01-01,아이폰 14,10,1000
4,2024-01-02,아이폰 14,20,1000
5,2024-01-02,갤럭시 S23,25,900
6,2024-01-02,아이패드 프로,10,800
7,2024-01-02,갤럭시 S23,5,900
8,2024-01-03,맥북 프로,8,2000
9,2024-01-03,아이폰 14,12,1000


In [26]:
df.pivot_table(
    index='날짜',
    columns='상품',
    values='판매량',
    fill_value=0,        # 결측치 대체값
    aggfunc='sum',       # 중복되는 값에 대한 집계함수
    margins=True         # 행 방향과 열 방향의 합계를 추가
)

상품,갤럭시 S23,맥북 프로,아이패드 프로,아이폰 14,All
날짜,,,,,
2024-01-01,30,0,5,25,60
2024-01-02,30,0,10,20,60
2024-01-03,0,8,0,12,20
All,60,8,15,57,140
